### Imports

In [1]:
import sys
sys.dont_write_bytecode = True


import torch
import numpy as np
import random
import os

def set_seeds(seed_value=42):
    """Sets seeds for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)

set_seeds(42) 

import json
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import warnings
import logging
from datetime import datetime
warnings.filterwarnings('ignore')

from model import get_model
from config import CFG
from dataset import get_dataset_class
from transform import get_transforms
from runner import run_baseline, run_lodo

torch.manual_seed(CFG["system"]["seed"])
np.random.seed(CFG["system"]["seed"])

device = CFG["system"]["device"]
print(f"Device: {device}")
print(f"PyTorch: {torch.__version__}")

DS = "PACS"
MODEL_NAME = "resnet18"

Skipping import of cpp extensions due to incompatible torch version 2.8.0+cu126 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info
W1029 12:11:24.891000 7228 torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


Device: cuda
PyTorch: 2.8.0+cu126


### DataLoading

In [2]:
train_transform, test_transform = get_transforms(img_size=224, augment=False, use_imagenet_norm=False)

DatasetClass = get_dataset_class(DS)

ld = DatasetClass(
    data_root=CFG["datasets"][DS]["root"],
    transform=train_transform,
    batch_size=CFG["train"]["batch_size"]
)

print("\nData loaders ready!")


Data loaders ready!


### Logging

In [3]:
dataset_name = DS
base_dir = os.path.join(os.getcwd(), "PACS_MISC2")
subdirs = ["logs", "checkpoints", "plots"]

for sub in subdirs:
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_file = os.path.join(base_dir, "logs", f"train_{timestamp}.log")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(f"{dataset_name}_logger")

logger.info(f"Initialized experiment directories for {dataset_name}")
logger.info(f"Logs: {os.path.join(base_dir, 'logs')}")
logger.info(f"Checkpoints: {os.path.join(base_dir, 'checkpoints')}")
logger.info(f"Plots: {os.path.join(base_dir, 'plots')}")

2025-10-29 12:11:26,198 | INFO | Initialized experiment directories for PACS
2025-10-29 12:11:26,198 | INFO | Logs: d:\Haseeb\SPROJ\GRQO\Vit-GRQO\resnet18\PACS_MISC2\logs
2025-10-29 12:11:26,198 | INFO | Checkpoints: d:\Haseeb\SPROJ\GRQO\Vit-GRQO\resnet18\PACS_MISC2\checkpoints
2025-10-29 12:11:26,198 | INFO | Plots: d:\Haseeb\SPROJ\GRQO\Vit-GRQO\resnet18\PACS_MISC2\plots


### Setup

In [9]:
domains = CFG["datasets"][DS]["domains"]
loaders = {d: {"train": ld.get_dataloader(d, train=True), "val": ld.get_dataloader(d, train=False)} for d in domains}
ckpt_root = os.path.join(base_dir, "checkpoints_Misc")
log_dir = os.path.join(base_dir, "logs_Misc")
plots_dir = os.path.join(base_dir, "plots_Misc")
os.makedirs(ckpt_root, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)
model_factory = lambda cfg, dataset_key: get_model(cfg,dataset=DS)
CFG["train"]["lr"] = 1e-4
optimizer_fn = lambda model: optim.AdamW(model.parameters(), lr=CFG["train"]["lr"], weight_decay=CFG["train"].get("weight_decay", 0.0))
CFG["train"]["epochs"] = 10
device = CFG["system"]["device"]
epochs = CFG["train"]["epochs"]

{
  "lodo_results": {
    "art_painting": 0.8341463414634146,
    "cartoon": 0.7974413646055437,
    "photo": 0.9580838323353293,
    "sketch": 0.6017811704834606
  },
  "timestamp": "20251004_020611"
}

### Leave One Domain Out

In [10]:
CFG["grqo"]["num_tokens"] = 48
CFG["grqo"]["random_k"] = 48
lodo_results, lodo_mean, lodo_summary = run_lodo(
    model_fn=model_factory,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    ckpt_root=ckpt_root,
    log_dir=log_dir,
    epochs=epochs
)

2025-10-29 12:35:27,100 | INFO | === LODO: Leaving out domain 'art_painting' ===



=== LODO: Leaving out domain 'art_painting' ===


Evaluating: 100%|██████████| 16/16 [00:03<00:00,  4.94it/s]
2025-10-29 12:35:48,565 | INFO | [art_painting] Epoch 1/10 | Train - Loss: 0.5708, Cls: 0.5613, GRQO: 0.0095, Acc: 0.8169 | Val - Loss: 0.7270, Cls: 0.7257, GRQO: 0.0013, Acc: 0.7710
2025-10-29 12:35:48,632 | INFO | [art_painting] New best val acc: 0.7710


[art_painting] Epoch 1/10 | Train - Loss: 0.5708, Cls: 0.5613, GRQO: 0.0095, Acc: 0.8169 | Val - Loss: 0.7270, Cls: 0.7257, GRQO: 0.0013, Acc: 0.7710
[art_painting] New best val acc: 0.7710


Evaluating: 100%|██████████| 16/16 [00:03<00:00,  4.52it/s]
2025-10-29 12:36:10,938 | INFO | [art_painting] Epoch 2/10 | Train - Loss: 0.1048, Cls: 0.1022, GRQO: 0.0026, Acc: 0.9655 | Val - Loss: 0.9646, Cls: 0.9640, GRQO: 0.0006, Acc: 0.7319


...

[sketch] Epoch 8/10 | Train - Loss: 0.0018, Cls: 0.0005, GRQO: 0.0012, Acc: 1.0000 | Val - Loss: 1.5232, Cls: 1.5230, GRQO: 0.0002, Acc: 0.6668


Evaluating: 100%|██████████| 31/31 [00:06<00:00,  4.66it/s]
2025-10-29 12:48:09,986 | INFO | [sketch] Epoch 9/10 | Train - Loss: 0.0016, Cls: 0.0005, GRQO: 0.0011, Acc: 1.0000 | Val - Loss: 1.4413, Cls: 1.4411, GRQO: 0.0002, Acc: 0.6801


[sketch] Epoch 9/10 | Train - Loss: 0.0016, Cls: 0.0005, GRQO: 0.0011, Acc: 1.0000 | Val - Loss: 1.4413, Cls: 1.4411, GRQO: 0.0002, Acc: 0.6801


Evaluating: 100%|██████████| 31/31 [00:07<00:00,  4.14it/s]
2025-10-29 12:48:29,777 | INFO | [sketch] Epoch 10/10 | Train - Loss: 0.0014, Cls: 0.0004, GRQO: 0.0010, Acc: 1.0000 | Val - Loss: 1.4575, Cls: 1.4573, GRQO: 0.0002, Acc: 0.6775
2025-10-29 12:48:29,777 | INFO | [sketch] Best Acc: 0.6867
2025-10-29 12:48:29,777 | INFO | ------------------------------------------------------------
2025-10-29 12:48:29,777 | INFO | LODO finished | Mean Acc: 0.7885 | Summary saved to d:\Haseeb\SPROJ\GRQO\Vit-GRQO\resnet18\PACS_MISC2\logs_Misc\lodo_summary_20251029_124829.json


[sketch] Epoch 10/10 | Train - Loss: 0.0014, Cls: 0.0004, GRQO: 0.0010, Acc: 1.0000 | Val - Loss: 1.4575, Cls: 1.4573, GRQO: 0.0002, Acc: 0.6775
[sketch] Best Acc: 0.6867
------------------------------------------------------------
LODO finished | Mean Acc: 0.7885
Summary saved to d:\Haseeb\SPROJ\GRQO\Vit-GRQO\resnet18\PACS_MISC2\logs_Misc\lodo_summary_20251029_124829.json


### Baseline

In [11]:
baseline_results, baseline_mean = run_baseline(
    model_name=MODEL_NAME,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    epochs=CFG["train"]["epochs"]
)

2025-10-29 12:53:05,111 | INFO | Initializing ResNet baseline: resnet18
2025-10-29 12:53:05,252 | INFO | === Baseline LODO: Leaving out domain 'art_painting' ===


Initializing ResNet baseline: resnet18

=== Baseline LODO: Leaving out domain 'art_painting' ===


2025-10-29 12:53:21,134 | INFO | [art_painting] Epoch 1/10 | Train - Loss: 0.5165, Acc: 0.8322 | Val Acc: 0.7358


[art_painting] Epoch 1/10 | Train - Loss: 0.5165, Acc: 0.8322 | Val Acc: 0.7358


2025-10-29 12:53:36,784 | INFO | [art_painting] Epoch 2/10 | Train - Loss: 0.1050, Acc: 0.9731 | Val Acc: 0.7524


...

[sketch] Epoch 8/10 | Train - Loss: 0.0017, Acc: 1.0000 | Val Acc: 0.6457


2025-10-29 13:03:44,522 | INFO | [sketch] Epoch 9/10 | Train - Loss: 0.0014, Acc: 1.0000 | Val Acc: 0.6317


[sketch] Epoch 9/10 | Train - Loss: 0.0014, Acc: 1.0000 | Val Acc: 0.6317


2025-10-29 13:04:02,717 | INFO | [sketch] Epoch 10/10 | Train - Loss: 0.0013, Acc: 1.0000 | Val Acc: 0.6238
2025-10-29 13:04:02,717 | INFO | [sketch] Best Val Acc: 0.6768
2025-10-29 13:04:02,717 | INFO | ------------------------------------------------------------
2025-10-29 13:04:02,717 | INFO | Baseline LODO (resnet18) finished | Mean Acc: 0.7856


[sketch] Epoch 10/10 | Train - Loss: 0.0013, Acc: 1.0000 | Val Acc: 0.6238
[sketch] Best Val Acc: 0.6768
------------------------------------------------------------
Baseline LODO (resnet18) finished | Mean Acc: 0.7856
